# Rebuild All Events
    This script builds/rebuilds all the events of Live Mouse Tracker
    You can also create your own events and add them to this process
    
    Approximate time is 1h of computation for a 24h database of 4 animals


# Parameters:
    This will compute automatically events for your database, and adjust settings for you.
    
    If you remove the automatic settings, you can force parameters:
        Set minT and maxT to process the database.
        Set windowT to divide the computation load in segment of that duration. Default value is 1 day.

In [3]:
import sys
sys.path.insert(1, "../")
from lmtanalysis.Measure import *

# set this to false if you want to set manual parameters.
automaticSettings = True

# Manual parameters:

''' minT and maxT to process the analysis (expressed in frame) '''
minT = 0
maxT = 3*oneDay
''' time window to compute the events (in frame). '''
windowT = 1*oneDay
''' speed up process '''
USE_CACHE_LOAD_DETECTION_CACHE = True

# Run this section to compute your database
    You will be prompt to provide a database

In [7]:
''' Created on 26 march 2019 @author: Fab '''

import sys
sys.path.insert(1, "../")
import sqlite3
from lmtanalysis.Animal import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventTrain2, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,BuildEventOralOralContact,\
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4, BuildEventOralGenitalContact, \
    BuildEventStop, BuildEventWaterPoint, \
    BuildEventMove, BuildEventGroup3MakeBreak, BuildEventGroup4MakeBreak,\
    BuildEventSideBySide, BuildEventSideBySideOpposite, BuildEventDetection,\
    BuildDataBaseIndex, BuildEventWallJump, BuildEventSAP,\
    BuildEventOralSideSequence, CheckWrongAnimal,\
    CorrectDetectionIntegrity, BuildEventNest4, BuildEventNest3, BuildEventFight, BuildEventGetAway
    
from psutil import virtual_memory

from tkinter.filedialog import askopenfilename
from lmtanalysis.TaskLogger import TaskLogger
import sys
import traceback
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.EventTimeLineCache import flushEventTimeLineCache,\
    disableEventTimeLineCache


from lmtanalysis.EventTimeLineCache import EventTimeLineCached

def getNumberOfFrames(file):
    
    connection = sqlite3.connect( file )
    c = connection.cursor() 
    query = "SELECT MAX(FRAMENUMBER) FROM FRAME";
    c.execute( query )
    result = c.fetchall()
    numberOfFrames = int( result[0][0] )
    connection.close()
    
    return numberOfFrames

class FileProcessException(Exception):
    pass

eventClassList = [                  
                  BuildEventOralOralContact,
                  BuildEventOralGenitalContact,
                  BuildEventSideBySide,
                  BuildEventSideBySideOpposite,
                  BuildEventTrain2,                  
                  BuildEventTrain3,
                  BuildEventTrain4,
                  BuildEventMove,
                  BuildEventFollowZone,
                  BuildEventRear5,
                  BuildEventSocialApproach,
                  BuildEventGetAway,
                  BuildEventSocialEscape,
                  BuildEventApproachRear,
                  BuildEventGroup2,
                  BuildEventGroup3,
                  BuildEventGroup4,
                  BuildEventGroup3MakeBreak,
                  BuildEventGroup4MakeBreak,
                  BuildEventStop,
                  BuildEventWaterPoint,
                  BuildEventApproachContact,
                  BuildEventWallJump,
                  BuildEventSAP,
                  BuildEventOralSideSequence,
                  BuildEventNest3,
                  BuildEventNest4
                   ]

def flushEvents( connection ):
    
    print("Flushing events...")

    for ev in eventClassList:
        
        chrono = Chronometer( "Flushing event " + str(ev) )
        ev.flush( connection );      
        chrono.printTimeInS()
    

def processTimeWindow( connection, currentMinT , currentMaxT ):
    
    CheckWrongAnimal.check( connection, tmin=currentMinT, tmax=currentMaxT )
    
    # Warning: enabling this process (CorrectDetectionIntegrity) will alter the database permanently
    # CorrectDetectionIntegrity.correct( connection, tmin=0, tmax=maxT )
                            
    BuildEventDetection.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT )

    animalPool = None
    
    flushEventTimeLineCache()
    
    if ( USE_CACHE_LOAD_DETECTION_CACHE ):
        print("Caching load of animal detection...")
        animalPool = AnimalPool( )
        animalPool.loadAnimals( connection )
        animalPool.loadDetection( start = currentMinT, end = currentMaxT )
        print("Caching load of animal detection done.")

    for ev in eventClassList:
        
        chrono = Chronometer( str( ev ) )
        ev.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT, pool = animalPool )        
        chrono.printTimeInS()

def process( file ):

    print(file)
    
    if automaticSettings:
        print("Automatic settings.")
        windowT = 1*oneDay
        minT = 0
        maxT = getNumberOfFrames( file )
        print ( "Auto max set to" , maxT , "frames")
        
    
    chronoFullFile = Chronometer("File " + file )
    
    connection = sqlite3.connect( file )
    
    BuildDataBaseIndex.buildDataBaseIndex( connection, force=False )
        
    currentT = minT

    try:

        flushEvents( connection )
        
        while currentT < maxT:
                        
            currentMinT = currentT
            currentMaxT = currentT+ windowT
            if ( currentMaxT > maxT ):
                currentMaxT = maxT
                
            chronoTimeWindowFile = Chronometer("File "+ file+ " currentMinT: "+ str(currentMinT)+ " currentMaxT: " + str(currentMaxT) );
            processTimeWindow( connection, currentMinT, currentMaxT )    
            chronoTimeWindowFile.printTimeInS()
            
            currentT += windowT

                        

        print("Full file process time: ")
        chronoFullFile.printTimeInS()
        
        TEST_WINDOWING_COMPUTATION = False
        
        if ( TEST_WINDOWING_COMPUTATION ):
                
            print("*************")
            print("************* TEST START SECTION")
            print("************* Test if results are the same with or without the windowing.")
            
            # display and record to a file all events found, checking with rolling idA from None to 4. Save nbEvent and total len
            
            eventTimeLineList = []
            
            eventList = getAllEvents( connection )
            file = open("outEvent"+str(windowT)+".txt","w")  
            file.write( "Event name\nnb event\ntotal duration" )
            
            for eventName in eventList:
                for idAnimalA in range( 0,5 ):                
                        idA = idAnimalA 
                        if idA == 0:
                            idA = None
                        timeLine = EventTimeLineCached( connection, file, eventName, idA,  minFrame=minT, maxFrame=maxT )
                        eventTimeLineList.append( timeLine )
                        file.write( timeLine.eventNameWithId+"\t"+str(len(timeLine.eventList))+"\t"+str(timeLine.getTotalLength())+"\n" )            
            
            file.close() 
    
            #plotMultipleTimeLine( eventTimeLineList )
            
            print("************* END TEST")
        
        
    except:
        
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        error = ''.join('!! ' + line for line in lines)
        
        t = TaskLogger( connection )
        t.addLog( error )
        
        print( error, file=sys.stderr ) 
        
        raise FileProcessException()
            
def getAllEvents( connection ):
    
    query = "select name from event group by name order by name"
    c = connection.cursor()     
    c.execute( query )    
    all_rows = c.fetchall()        
    header = [ "Name" ]
    data =[]    
    for row in all_rows:
        data.append( row[0] )
    return data        

print("Code launched.")

mem = virtual_memory()
availableMemoryGB = mem.total / 1000000000
print( "Total memory on computer: (GB)", availableMemoryGB ) 

if availableMemoryGB < 10:
    print( "Not enough memory to use cache load of events.")
    disableEventTimeLineCache()

print("A window is popping (maybe hidden) asking for files to process...")
files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

        flushEventTimeLineCache()

chronoFullBatch.printTimeInS()
print( "*** ALL JOBS DONE ***")

Code launched.
Total memory on computer: (GB) 17.078222848
A window is popping (maybe hidden) asking for files to process...
Processing file E:/lmt/base test/20170627_Experiment 1111 - with cache NEW.sqlite
E:/lmt/base test/20170627_Experiment 1111 - with cache NEW.sqlite
Automatic settings.
Auto max set to 2544046 frames
Creating lmtanalysis indexes...
CREATE INDEX `animalIndex` ON `ANIMAL` (`ID` );
index animalIndex already exists
CREATE INDEX `detectionIndex` ON `DETECTION` (`ID` ASC,`FRAMENUMBER` ASC);
index detectionIndex already exists
CREATE INDEX `detetIdIndex` ON `DETECTION` (`ID` ASC);
index detetIdIndex already exists
CREATE INDEX `detframenumberIndex` ON `DETECTION` (`FRAMENUMBER` ASC);
index detframenumberIndex already exists
CREATE INDEX `eventEndFrameIndex` ON `EVENT` (`ENDFRAME` ASC);
index eventEndFrameIndex already exists
CREATE INDEX `eventIndex` ON `EVENT` (`ID` ASC,`STARTFRAME` ASC,`ENDFRAME` ASC);
index eventIndex already exists
CREATE INDEX `eventStartFrameIndex`

Number of event deleted: 25986 
Deleting event Stop isolated (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Stop isolated"
Number of event deleted: 46291 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventStop' from '..\\lmtanalysis\\BuildEventStop.py'>  ]  11.81249451637268  seconds
Deleting event Water Zone (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Water Zone"
Number of event deleted: 3578 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventWaterPoint' from '..\\lmtanalysis\\BuildEventWaterPoint.py'>  ]  2.3281123638153076  seconds
Deleting event Approach contact (idA:None,idB:None,idC:None,idD:None) from base...
DELETE FROM EVENT WHERE NAME="Approach contact"
Number of event deleted: 54799 
[Chrono  Flushing event <module 'lmtanalysis.BuildEventApproachContact' from '..\\lmtanalysis\\BuildEventApproachContact.py'>  ]  7.109373569488525  seconds
Deleting event WallJump (idA:None,idB:None,idC:None,idD:No

Number of event:  5660
Mean length of event:  4.014664310954063
first event frame:  618
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 5660 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:1 idC:None idD:None loadEvent: False
Number of event:  5379
Mean length of event:  4.514407882506042
first event frame:  255
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 5379 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  3948
Mean length of event:  4.107396149949341
first event frame:  441
Keep previous entry.
Saving timeLine: Oral-oral Contact ( 3948 events )
Oral-oral Contact
Event Oral-oral Contact created. eventNameWithId = Oral-oral Contact idA:2 idB:4 idC:None idD:None loadEvent: False
Number of event:  4170
Mean length of event:  4.094244604316547
first event frame:  495
Keep previous entry.
Saving ti

Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:2 idB:3 idC:None idD:None loadEvent: False
Number of event:  3909
Mean length of event:  5.925044768482988
first event frame:  223
Keep previous entry.
Saving timeLine: Side by side Contact ( 3909 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:2 idB:4 idC:None idD:None loadEvent: False
Number of event:  4118
Mean length of event:  5.717095677513356
first event frame:  538
Keep previous entry.
Saving timeLine: Side by side Contact ( 4118 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by side Contact idA:3 idB:1 idC:None idD:None loadEvent: False
Number of event:  3659
Mean length of event:  4.555616288603444
first event frame:  269
Keep previous entry.
Saving timeLine: Side by side Contact ( 3659 events )
Side by side Contact
Event Side by side Contact created. eventNameWithId = Side by

Oral-genital Contact  Id( 3 , 2 , None , None ) Loaded ( 4032  records loaded in  4.031226873397827 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 4 , None , None ) Loaded ( 3882  records loaded in  3.984398603439331 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 1 , None , None ) Loaded ( 5319  records loaded in  4.031244993209839 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 2 , None , None ) Loaded ( 4081  records loaded in  4.312484502792358 S )
Caching eventTimeLine
Oral-genital Contact  Id( 4 , 3 , None , None ) Loaded ( 3762  records loaded in  3.984393358230591 S )
Caching eventTimeLine
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:2 idC:None idD:None loadEvent: False
Number of event:  56
Mean length of event:  9.357142857142858
first event frame:  2657
Keep previous entry.
Saving timeLine: Train2 ( 56 events )
Train2
Event Train2 created. eventNameWithId = Train2 idA:1 idB:3 idC:None idD:None loadEvent: False
Number of event:  87
M

...

Group4  Id( 2 , 1 , 4 , None ) Loaded ( 0  records loaded in  7.035839557647705 S )
Caching eventTimeLine
Group4  Id( 2 , 3 , 1 , None ) Loaded ( 0  records loaded in  7.046852111816406 S )
Caching eventTimeLine
Group4  Id( 2 , 3 , 4 , None ) Loaded ( 0  records loaded in  6.890648603439331 S )
Caching eventTimeLine
Group4  Id( 2 , 4 , 1 , None ) Loaded ( 0  records loaded in  6.923431873321533 S )
Caching eventTimeLine
Group4  Id( 2 , 4 , 3 , None ) Loaded ( 0  records loaded in  6.89064884185791 S )
Caching eventTimeLine
Group4  Id( 3 , 1 , 2 , None ) Loaded ( 0  records loaded in  6.874998092651367 S )
Caching eventTimeLine
Group4  Id( 3 , 1 , 4 , None ) Loaded ( 0  records loaded in  6.874979019165039 S )
Caching eventTimeLine
Group4  Id( 3 , 2 , 1 , None ) Loaded ( 0  records loaded in  6.921900510787964 S )
Caching eventTimeLine
Group4  Id( 3 , 2 , 4 , None ) Loaded ( 0  records loaded in  6.87499475479126 S )
Caching eventTimeLine
Group4  Id( 3 , 4 , 1 , None ) Loaded ( 0  recor

Number of event:  1524
Mean length of event:  90.74015748031496
first event frame:  120
Keep previous entry.
Saving timeLine: Water Zone ( 1524 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Water Point','0','2019-03-27 12:10:20','0','2544046' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventWaterPoint' from '..\\lmtanalysis\\BuildEventWaterPoint.py'>  ]  11.234379053115845  seconds
Contact  Id( 1 , 2 , None , None ) Loaded from cache ( 5888  records. )
Social approach  Id( 1 , 2 , None , None ) Loaded from cache ( 21673  records. )
Contact  Id( 1 , 3 , None , None ) Loaded from cache ( 6754  records. )
Social approach  Id( 1 , 3 , None , None ) Loaded from cache ( 19229  records. )
Contact  Id( 1 , 4 , None , None ) Loaded from cache ( 6062  records. )
Social approach  Id( 1 , 4 , None , None ) Loaded from cache ( 21516  records. )
Contact  Id( 2 , 1 , None , None ) Loaded from cache ( 5891  records. )
Social approach  Id( 2 , 1 , Non

Event SAP created. eventNameWithId = SAP idA:2 idB:None idC:None idD:None loadEvent: False
Number of event:  7776
Mean length of event:  4.237654320987654
first event frame:  97
Keep previous entry.
Saving timeLine: SAP ( 7776 events )
Event SAP created. eventNameWithId = SAP idA:4 idB:None idC:None idD:None loadEvent: False
Number of event:  9999
Mean length of event:  3.938793879387939
first event frame:  83
Keep previous entry.
Saving timeLine: SAP ( 9999 events )
INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event SAP','0','2019-03-27 12:12:52','0','2544046' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventSAP' from '..\\lmtanalysis\\BuildEventSAP.py'>  ]  28.109363317489624  seconds
Animal Id:1 Name:A RFID:000004065006 Genotype:KO User1:KO1
mean animal bodysize:  37.451591572945944
Oral-oral Contact  Id( 3 , 1 , None , None ) Loaded ( 4110  records loaded in  8.000011682510376 S )
Caching eventTimeLine
Oral-genital Contact  Id( 3 , 1 , None , N

Number of event:  179
Mean length of event:  61.452513966480446
first event frame:  16778
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 179 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:1 idB:3 idC:None idD:None loadEvent: False
seq oral geni - oral oral idA:1 idB:3 idC:None idD:None
Number of event:  113
Mean length of event:  47.150442477876105
first event frame:  16324
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 113 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:1 idB:4 idC:None idD:None loadEvent: False
seq oral geni - oral oral idA:1 idB:4 idC:None idD:None
Number of event:  131
Mean length of event:  51.961832061068705
first event frame:  40228
Keep previous entry.
Saving timeLine: seq oral geni - oral oral ( 131 events )
Event seq oral geni - oral oral created. eventNameWithId = seq oral geni - oral oral idA:2 idB:1 idC:None idD:None lo

INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Nest4','0','2019-03-27 12:19:03','0','2544046' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventNest4' from '..\\lmtanalysis\\BuildEventNest4.py'>  ]  54.8437557220459  seconds
[Chrono  File E:/lmt/base test/20170627_Experiment 1111 - with cache NEW.sqlite currentMinT: 0 currentMaxT: 2544046  ]  2926.60289978981  seconds
Full file process time: 
[Chrono  File E:/lmt/base test/20170627_Experiment 1111 - with cache NEW.sqlite  ]  3078.4183473587036  seconds
[Chrono  Full batch  ]  3078.8871138095856  seconds
*** ALL JOBS DONE ***
